In [1]:
## This file implements neural networks and random forest on p0040presabs_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 60.92% for over-sampling data, 65.52% after improvement.
## And it is 96.43% for combination data.
## For random forest, the accuracy is 58.62% for over-sampling data and 100% for combination data.
## For random forest with cross-validation, the mean accuracy is 60.77% for over-sampling, and 98.46% for combination data.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0040presabs_qual.csv')
df.shape

(255, 373)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
5      2
6      2
7      0
8      1
9      1
10     0
11     2
12     1
13     0
14     2
15     2
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     2
24     2
25     0
26     2
27     0
28     1
29     2
      ..
225    2
226    1
227    1
228    2
229    2
230    1
231    1
232    2
233    1
234    1
235    2
236    1
237    2
238    1
239    0
240    1
241    1
242    0
243    1
244    1
245    2
246    2
247    1
248    2
249    1
250    2
251    1
252    1
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

2    96
1    95
0    64
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,...,hxlA_2,group_8822,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno
0,107,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,2
1,109,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,115,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 372)

In [10]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,hxlA_2,group_8822,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 371) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 96), (1, 96), (2, 96)]


Using TensorFlow backend.


In [13]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 44), (1, 11), (2, 37)]


In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 201 samples, validate on 87 samples
Epoch 1/100
201/201 [==============================] - 0s 754us/step - loss: 1.0515 - accuracy: 0.4677 - val_loss: 1.0153 - val_accuracy: 0.4943
Epoch 2/100
201/201 [==============================] - 0s 72us/step - loss: 0.9040 - accuracy: 0.5522 - val_loss: 0.9822 - val_accuracy: 0.4828
Epoch 3/100
201/201 [==============================] - 0s 105us/step - loss: 0.8255 - accuracy: 0.6219 - val_loss: 0.9918 - val_accuracy: 0.4713
Epoch 4/100
201/201 [==============================] - 0s 133us/step - loss: 0.7945 - accuracy: 0.6567 - val_loss: 1.0093 - val_accuracy: 0.5057
Epoch 5/100
201/201 [==============================] - 0s 169us/step - loss: 0.7732 - accuracy: 0.6617 - val_loss: 1.0232 - val_accuracy: 0.4253
Epoch 6/100
201/201 [==============================] - 0s 159us/step - loss: 0.7404 - accuracy: 0.6716 - val_loss: 0.9931 - val_accuracy: 0.5402
Epoch 7/100
201/201 [==============================] - 0s 154us/step - loss: 0.7213 - 

201/201 [==============================] - 0s 142us/step - loss: 0.3253 - accuracy: 0.9104 - val_loss: 1.1323 - val_accuracy: 0.5632
Epoch 57/100
201/201 [==============================] - 0s 156us/step - loss: 0.3259 - accuracy: 0.9055 - val_loss: 1.0630 - val_accuracy: 0.6207
Epoch 58/100
201/201 [==============================] - 0s 109us/step - loss: 0.3102 - accuracy: 0.9104 - val_loss: 1.0652 - val_accuracy: 0.6207
Epoch 59/100
201/201 [==============================] - 0s 93us/step - loss: 0.3212 - accuracy: 0.8706 - val_loss: 1.1457 - val_accuracy: 0.5862
Epoch 60/100
201/201 [==============================] - 0s 97us/step - loss: 0.3127 - accuracy: 0.8756 - val_loss: 1.0656 - val_accuracy: 0.6207
Epoch 61/100
201/201 [==============================] - 0s 112us/step - loss: 0.3050 - accuracy: 0.9104 - val_loss: 1.0940 - val_accuracy: 0.6207
Epoch 62/100
201/201 [==============================] - 0s 124us/step - loss: 0.3001 - accuracy: 0.9005 - val_loss: 1.1049 - val_accuracy: 

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

87/87 [==============================] - 0s 87us/step
over-sampling test accuracy: 60.92%


In [45]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [46]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [47]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 201 samples, validate on 87 samples
Epoch 1/1000
201/201 [==============================] - 0s 617us/step - loss: 1.1186 - accuracy: 0.3781 - val_loss: 1.0485 - val_accuracy: 0.4138
Epoch 2/1000
201/201 [==============================] - 0s 113us/step - loss: 0.9885 - accuracy: 0.4428 - val_loss: 1.0271 - val_accuracy: 0.4713
Epoch 3/1000
201/201 [==============================] - 0s 122us/step - loss: 0.9330 - accuracy: 0.5373 - val_loss: 1.0156 - val_accuracy: 0.4828
Epoch 4/1000
201/201 [==============================] - 0s 87us/step - loss: 0.8823 - accuracy: 0.6020 - val_loss: 1.0016 - val_accuracy: 0.5287
Epoch 5/1000
201/201 [==============================] - 0s 136us/step - loss: 0.8452 - accuracy: 0.5970 - val_loss: 0.9941 - val_accuracy: 0.5287
Epoch 6/1000
201/201 [==============================] - 0s 121us/step - loss: 0.8084 - accuracy: 0.6318 - val_loss: 0.9890 - val_accuracy: 0.5057
Epoch 7/1000
201/201 [==============================] - 0s 112us/step - loss: 0.

201/201 [==============================] - 0s 114us/step - loss: 0.2444 - accuracy: 0.9303 - val_loss: 1.2069 - val_accuracy: 0.6437
Epoch 57/1000
201/201 [==============================] - 0s 137us/step - loss: 0.2301 - accuracy: 0.9104 - val_loss: 1.2897 - val_accuracy: 0.6437
Epoch 58/1000
201/201 [==============================] - 0s 88us/step - loss: 0.2223 - accuracy: 0.9353 - val_loss: 1.2950 - val_accuracy: 0.6207
Epoch 59/1000
201/201 [==============================] - 0s 134us/step - loss: 0.2141 - accuracy: 0.9403 - val_loss: 1.3458 - val_accuracy: 0.6437
Epoch 60/1000
201/201 [==============================] - 0s 101us/step - loss: 0.2071 - accuracy: 0.9403 - val_loss: 1.3205 - val_accuracy: 0.6552
Epoch 61/1000
201/201 [==============================] - 0s 104us/step - loss: 0.2007 - accuracy: 0.9453 - val_loss: 1.3549 - val_accuracy: 0.6207
Epoch 62/1000
201/201 [==============================] - 0s 108us/step - loss: 0.2026 - accuracy: 0.9353 - val_loss: 1.3007 - val_acc

Epoch 112/1000
201/201 [==============================] - 0s 93us/step - loss: 0.1088 - accuracy: 0.9701 - val_loss: 1.9553 - val_accuracy: 0.5977
Epoch 113/1000
201/201 [==============================] - 0s 93us/step - loss: 0.1172 - accuracy: 0.9652 - val_loss: 1.7307 - val_accuracy: 0.6552
Epoch 114/1000
201/201 [==============================] - 0s 106us/step - loss: 0.1093 - accuracy: 0.9701 - val_loss: 1.8185 - val_accuracy: 0.6552
Epoch 115/1000
201/201 [==============================] - 0s 111us/step - loss: 0.1056 - accuracy: 0.9652 - val_loss: 1.8500 - val_accuracy: 0.6552
Epoch 116/1000
201/201 [==============================] - 0s 96us/step - loss: 0.0987 - accuracy: 0.9751 - val_loss: 1.8895 - val_accuracy: 0.6437
Epoch 117/1000
201/201 [==============================] - 0s 89us/step - loss: 0.1040 - accuracy: 0.9701 - val_loss: 1.8896 - val_accuracy: 0.6552
Epoch 118/1000
201/201 [==============================] - 0s 98us/step - loss: 0.1121 - accuracy: 0.9602 - val_loss:

201/201 [==============================] - 0s 95us/step - loss: 0.0664 - accuracy: 0.9701 - val_loss: 2.1889 - val_accuracy: 0.6437
Epoch 168/1000
201/201 [==============================] - 0s 92us/step - loss: 0.0677 - accuracy: 0.9701 - val_loss: 2.2218 - val_accuracy: 0.6322
Epoch 169/1000
201/201 [==============================] - 0s 87us/step - loss: 0.0664 - accuracy: 0.9801 - val_loss: 2.1682 - val_accuracy: 0.6552
Epoch 170/1000
201/201 [==============================] - 0s 100us/step - loss: 0.0661 - accuracy: 0.9751 - val_loss: 2.1693 - val_accuracy: 0.6552
Epoch 171/1000
201/201 [==============================] - 0s 87us/step - loss: 0.0654 - accuracy: 0.9751 - val_loss: 2.2345 - val_accuracy: 0.6322
Epoch 172/1000
201/201 [==============================] - 0s 85us/step - loss: 0.0671 - accuracy: 0.9751 - val_loss: 2.2305 - val_accuracy: 0.6437
Epoch 173/1000
201/201 [==============================] - 0s 87us/step - loss: 0.0695 - accuracy: 0.9701 - val_loss: 2.1981 - val_ac

201/201 [==============================] - 0s 90us/step - loss: 0.0518 - accuracy: 0.9900 - val_loss: 2.4843 - val_accuracy: 0.6437
Epoch 223/1000
201/201 [==============================] - 0s 82us/step - loss: 0.0576 - accuracy: 0.9851 - val_loss: 2.4518 - val_accuracy: 0.6437
Epoch 224/1000
201/201 [==============================] - 0s 84us/step - loss: 0.0674 - accuracy: 0.9751 - val_loss: 2.5133 - val_accuracy: 0.6552
Epoch 225/1000
201/201 [==============================] - 0s 87us/step - loss: 0.0535 - accuracy: 0.9801 - val_loss: 2.5027 - val_accuracy: 0.6552
Epoch 226/1000
201/201 [==============================] - 0s 98us/step - loss: 0.0526 - accuracy: 0.9900 - val_loss: 2.4655 - val_accuracy: 0.6437
Epoch 227/1000
201/201 [==============================] - 0s 86us/step - loss: 0.0498 - accuracy: 0.9801 - val_loss: 2.4230 - val_accuracy: 0.6437
Epoch 228/1000
201/201 [==============================] - 0s 103us/step - loss: 0.0521 - accuracy: 0.9801 - val_loss: 2.5139 - val_ac

...

201/201 [==============================] - 0s 90us/step - loss: 0.0333 - accuracy: 0.9801 - val_loss: 4.0402 - val_accuracy: 0.6667
Epoch 778/1000
201/201 [==============================] - 0s 108us/step - loss: 0.0258 - accuracy: 0.9900 - val_loss: 4.2137 - val_accuracy: 0.6667
Epoch 779/1000
201/201 [==============================] - 0s 93us/step - loss: 0.0230 - accuracy: 0.9900 - val_loss: 3.9203 - val_accuracy: 0.6437
Epoch 780/1000
201/201 [==============================] - 0s 85us/step - loss: 0.0278 - accuracy: 0.9900 - val_loss: 4.0100 - val_accuracy: 0.6667
Epoch 781/1000
201/201 [==============================] - 0s 107us/step - loss: 0.0239 - accuracy: 0.9900 - val_loss: 4.0512 - val_accuracy: 0.6667
Epoch 782/1000
201/201 [==============================] - 0s 90us/step - loss: 0.0232 - accuracy: 0.9900 - val_loss: 3.9920 - val_accuracy: 0.6667
Epoch 783/1000
201/201 [==============================] - 0s 89us/step - loss: 0.0243 - accuracy: 0.9900 - val_loss: 4.0186 - val_a

Epoch 833/1000
201/201 [==============================] - 0s 90us/step - loss: 0.0415 - accuracy: 0.9851 - val_loss: 4.3277 - val_accuracy: 0.6552
Epoch 834/1000
201/201 [==============================] - 0s 87us/step - loss: 0.0346 - accuracy: 0.9900 - val_loss: 4.4933 - val_accuracy: 0.6552
Epoch 835/1000
201/201 [==============================] - 0s 86us/step - loss: 0.0311 - accuracy: 0.9900 - val_loss: 4.0815 - val_accuracy: 0.6667
Epoch 836/1000
201/201 [==============================] - 0s 89us/step - loss: 0.0246 - accuracy: 0.9900 - val_loss: 3.9533 - val_accuracy: 0.6667
Epoch 837/1000
201/201 [==============================] - 0s 94us/step - loss: 0.0519 - accuracy: 0.9751 - val_loss: 3.9472 - val_accuracy: 0.6552
Epoch 838/1000
201/201 [==============================] - 0s 91us/step - loss: 0.0504 - accuracy: 0.9801 - val_loss: 4.0000 - val_accuracy: 0.6437
Epoch 839/1000
201/201 [==============================] - 0s 87us/step - loss: 0.0335 - accuracy: 0.9900 - val_loss: 4

201/201 [==============================] - 0s 91us/step - loss: 0.0236 - accuracy: 0.9900 - val_loss: 4.1660 - val_accuracy: 0.6437
Epoch 889/1000
201/201 [==============================] - 0s 88us/step - loss: 0.0257 - accuracy: 0.9900 - val_loss: 4.1516 - val_accuracy: 0.6437
Epoch 890/1000
201/201 [==============================] - 0s 92us/step - loss: 0.0219 - accuracy: 0.9900 - val_loss: 4.2437 - val_accuracy: 0.6437
Epoch 891/1000
201/201 [==============================] - 0s 89us/step - loss: 0.0263 - accuracy: 0.9900 - val_loss: 4.2606 - val_accuracy: 0.6437
Epoch 892/1000
201/201 [==============================] - 0s 84us/step - loss: 0.0246 - accuracy: 0.9900 - val_loss: 4.1979 - val_accuracy: 0.6552
Epoch 893/1000
201/201 [==============================] - 0s 88us/step - loss: 0.0257 - accuracy: 0.9900 - val_loss: 4.2213 - val_accuracy: 0.6322
Epoch 894/1000
201/201 [==============================] - 0s 99us/step - loss: 0.0245 - accuracy: 0.9900 - val_loss: 4.3644 - val_acc

Epoch 944/1000
201/201 [==============================] - 0s 88us/step - loss: 0.0218 - accuracy: 0.9851 - val_loss: 4.4572 - val_accuracy: 0.6322
Epoch 945/1000
201/201 [==============================] - 0s 84us/step - loss: 0.0458 - accuracy: 0.9851 - val_loss: 4.3801 - val_accuracy: 0.6322
Epoch 946/1000
201/201 [==============================] - 0s 86us/step - loss: 0.0327 - accuracy: 0.9851 - val_loss: 4.3322 - val_accuracy: 0.6552
Epoch 947/1000
201/201 [==============================] - 0s 99us/step - loss: 0.0310 - accuracy: 0.9900 - val_loss: 4.6897 - val_accuracy: 0.6322
Epoch 948/1000
201/201 [==============================] - 0s 95us/step - loss: 0.0306 - accuracy: 0.9851 - val_loss: 4.3345 - val_accuracy: 0.6207
Epoch 949/1000
201/201 [==============================] - 0s 95us/step - loss: 0.0235 - accuracy: 0.9900 - val_loss: 4.2165 - val_accuracy: 0.6322
Epoch 950/1000
201/201 [==============================] - 0s 88us/step - loss: 0.0264 - accuracy: 0.9851 - val_loss: 4

Epoch 1000/1000
201/201 [==============================] - 0s 88us/step - loss: 0.0249 - accuracy: 0.9900 - val_loss: 4.4009 - val_accuracy: 0.6552


In [48]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

87/87 [==============================] - 0s 110us/step
over-sampling test accuracy: 65.52%


In [26]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [27]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 64 samples, validate on 28 samples
Epoch 1/100
64/64 [==============================] - 0s 2ms/step - loss: 1.0497 - accuracy: 0.4219 - val_loss: 0.8550 - val_accuracy: 0.5714
Epoch 2/100
64/64 [==============================] - 0s 113us/step - loss: 0.7890 - accuracy: 0.6719 - val_loss: 0.6874 - val_accuracy: 0.8571
Epoch 3/100
64/64 [==============================] - 0s 123us/step - loss: 0.6230 - accuracy: 0.8594 - val_loss: 0.5693 - val_accuracy: 0.8571
Epoch 4/100
64/64 [==============================] - 0s 211us/step - loss: 0.5177 - accuracy: 0.8750 - val_loss: 0.4789 - val_accuracy: 0.8571
Epoch 5/100
64/64 [==============================] - 0s 251us/step - loss: 0.4371 - accuracy: 0.8750 - val_loss: 0.4137 - val_accuracy: 0.8571
Epoch 6/100
64/64 [==============================] - 0s 142us/step - loss: 0.3817 - accuracy: 0.8750 - val_loss: 0.3683 - val_accuracy: 0.9286
Epoch 7/100
64/64 [==============================] - 0s 137us/step - loss: 0.3394 - accuracy: 0.9219

64/64 [==============================] - 0s 129us/step - loss: 0.0444 - accuracy: 1.0000 - val_loss: 0.1120 - val_accuracy: 0.9643
Epoch 58/100
64/64 [==============================] - 0s 127us/step - loss: 0.0442 - accuracy: 1.0000 - val_loss: 0.1092 - val_accuracy: 0.9643
Epoch 59/100
64/64 [==============================] - 0s 180us/step - loss: 0.0424 - accuracy: 1.0000 - val_loss: 0.1075 - val_accuracy: 0.9643
Epoch 60/100
64/64 [==============================] - 0s 132us/step - loss: 0.0412 - accuracy: 1.0000 - val_loss: 0.1060 - val_accuracy: 0.9643
Epoch 61/100
64/64 [==============================] - 0s 147us/step - loss: 0.0400 - accuracy: 1.0000 - val_loss: 0.1047 - val_accuracy: 0.9643
Epoch 62/100
64/64 [==============================] - 0s 118us/step - loss: 0.0391 - accuracy: 1.0000 - val_loss: 0.1029 - val_accuracy: 0.9643
Epoch 63/100
64/64 [==============================] - 0s 150us/step - loss: 0.0379 - accuracy: 1.0000 - val_loss: 0.1016 - val_accuracy: 0.9643
Epoch

In [29]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

28/28 [==============================] - 0s 44us/step
combination test accuracy: 96.43%


In [30]:
#### add dropout and regularizor
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [31]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 64 samples, validate on 28 samples
Epoch 1/100
64/64 [==============================] - 0s 2ms/step - loss: 3.2833 - accuracy: 0.5781 - val_loss: 0.9904 - val_accuracy: 0.8571
Epoch 2/100
64/64 [==============================] - 0s 133us/step - loss: 4.7094 - accuracy: 0.6094 - val_loss: 0.8708 - val_accuracy: 0.8571
Epoch 3/100
64/64 [==============================] - 0s 146us/step - loss: 3.1588 - accuracy: 0.7500 - val_loss: 0.7852 - val_accuracy: 0.8571
Epoch 4/100
64/64 [==============================] - 0s 134us/step - loss: 2.4814 - accuracy: 0.7812 - val_loss: 0.7296 - val_accuracy: 0.8571
Epoch 5/100
64/64 [==============================] - 0s 137us/step - loss: 2.7203 - accuracy: 0.7812 - val_loss: 0.7069 - val_accuracy: 0.8571
Epoch 6/100
64/64 [==============================] - 0s 177us/step - loss: 3.7270 - accuracy: 0.6875 - val_loss: 0.6939 - val_accuracy: 0.8571
Epoch 7/100
64/64 [==============================] - 0s 164us/step - loss: 3.7210 - accuracy: 0.7500

Epoch 58/100
64/64 [==============================] - 0s 280us/step - loss: 2.1448 - accuracy: 0.6875 - val_loss: 0.8694 - val_accuracy: 0.9286
Epoch 59/100
64/64 [==============================] - 0s 174us/step - loss: 2.0661 - accuracy: 0.7656 - val_loss: 0.8657 - val_accuracy: 0.9286
Epoch 60/100
64/64 [==============================] - 0s 186us/step - loss: 1.8742 - accuracy: 0.8281 - val_loss: 0.8610 - val_accuracy: 0.9286
Epoch 61/100
64/64 [==============================] - 0s 135us/step - loss: 1.8041 - accuracy: 0.8438 - val_loss: 0.8540 - val_accuracy: 0.9643
Epoch 62/100
64/64 [==============================] - 0s 176us/step - loss: 1.1871 - accuracy: 0.8438 - val_loss: 0.8497 - val_accuracy: 0.9643
Epoch 63/100
64/64 [==============================] - 0s 201us/step - loss: 2.5405 - accuracy: 0.6875 - val_loss: 0.8504 - val_accuracy: 0.9643
Epoch 64/100
64/64 [==============================] - ETA: 0s - loss: 1.7954 - accuracy: 0.87 - 0s 175us/step - loss: 1.7303 - accuracy:

In [33]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

28/28 [==============================] - 0s 70us/step
combination test accuracy: 96.43%


In [34]:
############## Random Forest ##############

In [35]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [36]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 58.62%


In [37]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [38]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [39]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.54761905 0.61904762 0.58974359 0.61538462 0.66666667]
0.6076923076923076


In [40]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         0.92307692 1.         1.         1.        ]
0.9846153846153847
